In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
# https://www.kaggle.com/datasets/gmkeshav/tetuan-city-power-consumption
data = pd.read_csv('assets/power_consumption.csv')
# dimensão dos dados
data.shape

In [ ]:
data.head()

In [ ]:
# agregar consumo numa só coluna
data.set_index('DateTime', inplace=True)
data['load'] = data['Zone 1 Power Consumption'] + data['Zone 2  Power Consumption'] + data['Zone 3  Power Consumption']

In [ ]:
# transformar coluna de consumo como série
series = data['load']
series.index = pd.to_datetime(series.index)

series.head()

In [ ]:
series.plot()

In [ ]:
# agregar para granularidade horária
series_h = series.resample('H').sum()
series_h.plot()

In [ ]:
print(series_h.shape)
series_h.head()

## Múltiplas sazonalidades

In [ ]:
# analisar auto-correlação para verificar os diferentes padrões sazonais
from pmdarima.arima import nsdiffs, ndiffs
from pmdarima.utils import plot_acf

In [ ]:
plt = plot_acf(series_h, lags=48)

In [ ]:
series_d = series_h.diff(24)[24:]

plt = plot_acf(series_d, lags=48)

In [ ]:
plt = plot_acf(series_d, lags=24 * 7 + 1)

In [ ]:
series_d2 = series_d.diff(24*7)[(24*7):]

plt = plot_acf(series_d2, lags=24*7 + 1)

In [ ]:
plot_acf(series_d.diff(periods=24*7*4).dropna(), lags=24*7*4+1)

## Abordagens

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(series_h, test_size=48, shuffle=False)

### SARIMA

In [ ]:
from pmdarima import auto_arima

arima_model = auto_arima(train.tail(100), seasonal=True, m=24, trace=True)
y_arima_forecast = arima_model.predict(n_periods=48)

print(np.mean(abs(test - y_arima_forecast)))

### TBATS

In [ ]:
from tbats import TBATS

# treinar o modelo
estimator = TBATS(seasonal_periods=(24, 24 * 7, 24 * 7 * 4))
model = estimator.fit(train.tail(100))
# Forecast 48 days ahead
y_tbats_forecast = model.forecast(steps=48)

print(np.mean(abs(test - y_tbats_forecast)))

### Prophet

In [ ]:
from prophet import Prophet

model = Prophet(weekly_seasonality=True,
                daily_seasonality=True,
                yearly_seasonality=True)

# é possível adicionar sazonalidades específicas
model = model.add_seasonality(name='monthly', period=24 * 7 * 4, fourier_order=10)


train_df = train.reset_index()
train_df.columns = ['ds', 'y']

#model.fit(train_df.tail(1000))
model.fit(train_df)

y_prophet_forecast = model.make_future_dataframe(periods=48,
                                       include_history=False,
                                       freq='H')
y_prophet_forecast = model.predict(y_prophet_forecast)


In [ ]:
fig1 = model.plot(y_prophet_forecast)

In [ ]:
fig2 = model.plot_components(y_prophet_forecast)

In [ ]:
from prophet.plot import add_changepoints_to_plot

fig = model.plot(y_prophet_forecast)
a = add_changepoints_to_plot(fig.gca(), model, y_prophet_forecast)

In [ ]:
print(np.mean(abs(test - y_prophet_forecast['yhat'].values)))

### Optimizing Prophet

In [ ]:
from sklearn.model_selection import ParameterGrid, train_test_split
from sklearn.metrics import mean_absolute_error as mae
from src.prophet import optimize_prophet

params_grid = {'seasonality_mode': ['multiplicative', 'additive'],
               'growth': ['linear', 'flat'],
               'changepoint_prior_scale': [0.01,0.1, 0.25, 0.5],
               'seasonality_prior_scale': [0.01,0.5, 1, 5, 10],
               'n_changepoints': [0, 1, 2, 10]}


grid = ParameterGrid(params_grid)

train_in, validation = train_test_split(train, test_size=48, shuffle=False)

train_in_df = train_in.reset_index()
train_in_df.columns = ['ds', 'y']

val_results = {'losses': [], 'params': []}
for params in grid:
    print(params)
    model = Prophet(seasonality_mode=params['seasonality_mode'],
                    growth=params['growth'],
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    yearly_seasonality=False,
                    n_changepoints=params['n_changepoints'],
                    changepoint_prior_scale=params['changepoint_prior_scale'])
    
    model = model.fit(train_in_df)

    forecast = model.make_future_dataframe(periods=48,
                                           include_history=False,
                                           freq='H')
    forecast = model.predict(forecast)
    forecast = forecast.filter(items=['ds', 'yhat'])

    loss = mae(validation, forecast['yhat'].values)

    val_results['losses'].append(loss)
    val_results['params'].append(params)

idx_params = np.argmin(val_results['losses'])
params = val_results['params'][idx_params]

In [ ]:
params

In [ ]:
model = Prophet(weekly_seasonality=True,
                daily_seasonality=True,
                yearly_seasonality=False,
                **params)

model = model.add_seasonality(name='monthly', period=24 * 7 * 4, fourier_order=10)

model.fit(train_df)

y_prophetopt_forecast = model.make_future_dataframe(periods=test.shape[0],
                                       include_history=False,
                                       freq='H')
y_prophetopt_forecast = model.predict(y_prophetopt_forecast)
y_prophetopt_forecast = y_prophet_forecast.filter(items=['ds', 'yhat'])

print(np.mean(abs(test - y_prophetopt_forecast['yhat'].values)))

#### Feriados

In [ ]:
feriados = pd.DataFrame({
  'holiday': 'feriados',
  'ds': pd.to_datetime(['2017-01-13', '2017-02-07', '2017-03-19',
                        '2017-04-02', '2017-05-11', '2017-05-17',
                        '2017-09-24', '2017-10-07']),
  'lower_window': 0,
  'upper_window': 1,
})

model = Prophet(holidays=feriados,
                weekly_seasonality=True,
                daily_seasonality=True,
                yearly_seasonality=True)

model = model.add_seasonality(name='monthly', period=24 * 7 * 4, fourier_order=10)

model.fit(train_df)

y_prophet_forecast = model.make_future_dataframe(periods=48,
                                       include_history=False,
                                       freq='H')
y_prophet_forecast = model.predict(y_prophet_forecast)

fig = model.plot(y_prophet_forecast)
a = add_changepoints_to_plot(fig.gca(), model, y_prophet_forecast)

### Fourier and Repeating Basis Functions

In [ ]:
from src.tde import time_delay_embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import RidgeCV, Lasso

train_df = time_delay_embedding(train, n_lags=24, horizon=48)
train_df = train_df.dropna()
target_columns = train_df.columns.str.contains('\+')
X = train_df.iloc[:, ~target_columns]
Y = train_df.iloc[:, target_columns]

train_df.head()

In [ ]:
test_dummy = pd.Series(np.repeat(np.nan, 48), index=test.index)
test_df = time_delay_embedding(pd.concat([train.tail(24), test_dummy]), n_lags=24, horizon=48)
test_df = test_df.loc[test.head(1).index, :]
X_test = test_df.iloc[:, ~target_columns]
X_test

In [ ]:
from src.seasonality import FourierTerms

daily_terms = FourierTerms(period=24, n_terms=2, prefix='D_')
weekly_terms = FourierTerms(period=24 * 7, n_terms=2, prefix='W_')
monthly_terms = FourierTerms(period=24 * 30, n_terms=2, prefix='M_')

daily_fourier = daily_terms.transform(train_df.index)
weekly_fourier = weekly_terms.transform(train_df.index)
monthly_fourier = monthly_terms.transform(train_df.index)

weekly_fourier['MONTH'] = train_df.index.month

#daily_fourier.tail(400).plot(figsize=(15,6))
weekly_fourier.plot(figsize=(15,6))

In [ ]:
from src.seasonality import RBFTerms

rbf_hour = RBFTerms(n_terms=12, period='.hour', prefix='H_')
rbf_day = RBFTerms(n_terms=12, period='.day', prefix='D_')

rbf_hour.fit(train_df.index)
rbf_day.fit(train_df.index)

rbf_hour_x = rbf_hour.transform(train_df.index)
rbf_day_x = rbf_day.transform(train_df.index)

rbf_day_x.iloc[:,2:4].tail(1400).plot(figsize=(15,6))

In [ ]:
from src.seasonality import FourierTerms, RBFTerms

daily_terms = FourierTerms(period=24, n_terms=2, prefix='D_')
weekly_terms = FourierTerms(period=24 * 7, n_terms=2, prefix='W_')
monthly_terms = FourierTerms(period=24 * 30, n_terms=2, prefix='M_')

rbf_hour = RBFTerms(n_terms=12, period='.hour', prefix='H_')
rbf_day = RBFTerms(n_terms=12, period='.day', prefix='D_')

rbf_hour.fit(train_df.index)
rbf_day.fit(train_df.index)

rbf_hour_x = rbf_hour.transform(train_df.index)
rbf_day_x = rbf_day.transform(train_df.index)
daily_fourier = daily_terms.transform(train_df.index)
weekly_fourier = weekly_terms.transform(train_df.index)
monthly_fourier = monthly_terms.transform(train_df.index)

X_exog = pd.concat([X, daily_fourier, weekly_fourier, monthly_fourier,
                    rbf_hour_x, rbf_day_x],
                   ignore_index=False, axis=1)

model_with_terms = RandomForestRegressor()
model_with_terms.fit(X_exog, Y)
model_without_terms = RandomForestRegressor()
model_without_terms.fit(X, Y)



daily_ts_fourier = daily_terms.transform(X_test.index)
weekly_ts_fourier = weekly_terms.transform(X_test.index)
monthly_ts_fourier = monthly_terms.transform(X_test.index)
rbf_hour_ts = rbf_hour.transform(X_test.index)
rbf_day_ts = rbf_day.transform(X_test.index)

X_ts_exog = pd.concat([X_test, daily_ts_fourier,
                       weekly_ts_fourier, monthly_ts_fourier,
                       rbf_hour_ts, rbf_day_ts
                       ],
                      ignore_index=False, axis=1)

preds_with_terms = model_with_terms.predict(X_ts_exog)[0]
preds_without_terms = model_without_terms.predict(X_test)[0]

print(np.mean(abs(test - preds_with_terms)))
print(np.mean(abs(test - preds_without_terms)))

In [ ]:
pd.Series(dict(zip(X_exog.columns,model_with_terms.feature_importances_))).plot.barh(figsize=(8,13))

In [ ]:
results = {'SARIMA': np.mean(abs(test - y_arima_forecast)),
'TBATS': np.mean(abs(test - y_tbats_forecast)),
'PROPHET': np.mean(abs(test - y_prophet_forecast['yhat'].values)),
'PROPHET_OPT': np.mean(abs(test - y_prophetopt_forecast['yhat'].values)),
'RF+Fourier': np.mean(abs(test - preds_with_terms)),
'RF': np.mean(abs(test - preds_without_terms))
}

pd.Series(results).plot.bar()